## ENTREGA TP GRUPAL - PEIA - 19Cohorte

### INTEGRANTES

- Jose Perez
- Gustavo Ramoscelli
- Matias Tripode

In [ ]:
# importar librerías necesarias para la realización de la asignación

import pandas as pd
import numpy as np


In [21]:
# leer los datos del xlsx

archivo = 'Datos_examen_final_19Co2024_Grupo3.xlsx'
xlsx = pd.read_excel(archivo)
sheets = ['Santa Ana', 'La Floresta', 'Los Cedros', 'Palermo', 'Córdoba']

data = {sheet: pd.read_excel(archivo, sheet_name=sheet) for sheet in sheets}

In [22]:
# cambiar de formato de fecha la columna "Fecha"

for df in data.values():
    df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')
    df['Mes'] = df['Fecha'].dt.month

1. Determinen intervalos de confianza emp´ıricos para el supermercado ’Santa Ana’ en cada mes, para significancias del 95 % y el 99 %.

In [48]:
santa_ana = data['Santa Ana']

confidence_intervals = {}
alpha_values = [0.05, 0.01]
for alpha in alpha_values:
    intervals = {}
    for month in santa_ana['Mes'].unique():
        monthly_data = santa_ana[santa_ana['Mes'] == month]['Ventas']  # Filtrar datos por mes
        n = len(monthly_data)  # Número de observaciones
        mean = monthly_data.mean()  # Media muestral
        std_dev = monthly_data.std()  # Desviación estándar muestral
        std_err = std_dev / np.sqrt(n)  # Error estándar de la media
        
        # Valor crítico t (manual, usando np.percentile para aproximación)
        t_critical = np.percentile(np.random.standard_t(df=n - 1, size=1000000), [100 * (1 - alpha / 2)])[0]
        
        # Cálculo del intervalo
        ci_lower = mean - t_critical * std_err
        ci_upper = mean + t_critical * std_err
        intervals[month] = (ci_lower, ci_upper)
    confidence_intervals[alpha] = intervals

print("1. Intervalos de confianza para el supermercado 'Santa Ana':")
for alpha, intervals in confidence_intervals.items():
    print(f"\nNivel de significancia: {1-alpha:.0%}")
    for month, ci in intervals.items():
        print(f"  Mes {month}: Intervalo ({ci[0]:.2f}, {ci[1]:.2f})")

1. Intervalos de confianza para el supermercado 'Santa Ana':

Nivel de significancia: 95%
  Mes 1: Intervalo (16967.00, 18798.25)
  Mes 2: Intervalo (19322.11, 21657.34)
  Mes 3: Intervalo (21402.52, 23397.62)
  Mes 4: Intervalo (19710.43, 21896.29)
  Mes 5: Intervalo (21304.57, 22861.87)
  Mes 6: Intervalo (22969.87, 24677.59)
  Mes 7: Intervalo (20729.87, 22772.57)
  Mes 8: Intervalo (22413.99, 24540.78)
  Mes 9: Intervalo (22370.09, 23857.72)
  Mes 10: Intervalo (22202.88, 23961.88)
  Mes 11: Intervalo (21617.80, 23730.47)
  Mes 12: Intervalo (19625.12, 21363.70)

Nivel de significancia: 99%
  Mes 1: Intervalo (16644.92, 19120.33)
  Mes 2: Intervalo (18916.09, 22063.36)
  Mes 3: Intervalo (21053.26, 23746.88)
  Mes 4: Intervalo (19329.55, 22277.17)
  Mes 5: Intervalo (21032.22, 23134.22)
  Mes 6: Intervalo (22670.37, 24977.09)
  Mes 7: Intervalo (20374.28, 23128.16)
  Mes 8: Intervalo (22040.44, 24914.34)
  Mes 9: Intervalo (22109.77, 24118.04)
  Mes 10: Intervalo (21894.57, 24270.1

2. Realicen pruebas ANOVA para determinar si las ventas esperadas de todas las tiendas son iguales o no, con significancia del 95

In [47]:
# Combina las ventas de todas las tiendas en un único DataFrame, etiquetando por tienda
ventas_data = pd.concat([df[['Fecha', 'Ventas']].assign(Tienda=tienda) for tienda, df in data.items()])

# Cálculo de sumas de cuadrados
overall_mean = ventas_data['Ventas'].mean()  # Media global
SCE = sum(len(df) * (df['Ventas'].mean() - overall_mean) ** 2 for df in data.values())  # Suma de cuadrados entre grupos
SCD = sum(((df['Ventas'] - df['Ventas'].mean()) ** 2).sum() for df in data.values())  # Suma de cuadrados dentro de grupos

# Grados de libertad
gl_entre = len(data) - 1  # Número de grupos - 1
gl_dentro = len(ventas_data) - len(data)  # Total de observaciones - número de grupos

# Estadístico F
MS_entre = SCE / gl_entre  # Media cuadrática entre grupos
MS_dentro = SCD / gl_dentro  # Media cuadrática dentro de grupos
F_stat = MS_entre / MS_dentro  # Estadístico F

# Valor crítico F (manual, usando aproximación de distribución F)
alpha = 0.05  # Nivel de significancia
f_critical = np.percentile(np.random.f(gl_entre, gl_dentro, size=100), 100 * (1 - alpha))

# Simular valores de la distribución t con los grados de libertad calculados
t_distribution = np.random.standard_t(df, size=100)

# Proporción de valores mayores al valor absoluto de t_stat
p_value = 2 * np.mean(np.abs(t_distribution) > np.abs(F_stat))

print("\n2. Prueba ANOVA:")
print(f"  Estadístico F: {F_stat:.2f}")
print(f"  Valor crítico F: {f_critical:.2f}")
print(f"  P-valor: {p_value:.4e}")  

if F_stat > f_critical:
    print("  Las diferencias en las medias de las ventas entre las tiendas SON significativas.")
else:
    print("  No hay evidencia de diferencias significativas entre las medias de las ventas de las tiendas.")



2. Prueba ANOVA:
  Estadístico F: 114.45
  Valor crítico F: 2.06
  P-valor: 0.0000e+00
  Las diferencias en las medias de las ventas entre las tiendas SON significativas.


3.  Identifiquen la tienda con mayor promedio de ventas y la tienda con menor promedio de ventas y realicen una prueba de hip´otesis para determinar si la diferencia entre ellas es distinta de cero o no

In [46]:
mean_sales = {tienda: df['Ventas'].mean() for tienda, df in data.items()}
max_store = max(mean_sales, key=mean_sales.get)
min_store = min(mean_sales, key=mean_sales.get)

# Prueba de hipótesis entre tienda con mayor y menor promedio
max_data = data[max_store]['Ventas']
min_data = data[min_store]['Ventas']

# Estadístico t manual
n1, n2 = len(max_data), len(min_data)
mean1, mean2 = max_data.mean(), min_data.mean()
std1, std2 = max_data.std(), min_data.std()

# Error estándar combinado
pooled_std = np.sqrt(((n1 - 1) * std1 ** 2 + (n2 - 1) * std2 ** 2) / (n1 + n2 - 2))
std_err_diff = pooled_std * np.sqrt(1 / n1 + 1 / n2)

# Cálculo de t
t_stat = (mean1 - mean2) / std_err_diff

# Grados de libertad
df = n1 + n2 - 2

# Simular valores de la distribución t con los grados de libertad calculados
t_distribution = np.random.standard_t(df, size=1000000)

# Proporción de valores mayores al valor absoluto de t_stat
p_value = 2 * np.mean(np.abs(t_distribution) > np.abs(t_stat))

print("\n3. Promedios de ventas por tienda:")
for tienda, promedio in mean_sales.items():
    print(f"  {tienda}: {promedio:.2f}")

print(f"  Tienda con mayor promedio: {max_store} ({mean_sales[max_store]:.2f})")
print(f"  Tienda con menor promedio: {min_store} ({mean_sales[min_store]:.2f})")
print(f"  Estadístico t: {t_stat:.2f}")
print(f"  P-valor: {p_value:.4e}")  
if p_value < 0.05:
    print("  Las diferencias entre las tiendas son significativas.")
else:
    print("  No hay evidencia suficiente para afirmar que las diferencias entre las tiendas son significativas.")


3. Promedios de ventas por tienda:
  Santa Ana: 21842.40
  La Floresta: 18049.11
  Los Cedros: 19713.10
  Palermo: 18851.66
  Córdoba: 21784.30
  Tienda con mayor promedio: Santa Ana (21842.40)
  Tienda con menor promedio: La Floresta (18049.11)
  Estadístico t: 16.23
  P-valor: 0.0000e+00
  Las diferencias entre las tiendas son significativas.
